In [ ]:
# !pip install transformers langchain chromadb mysql-connector-python

In [1]:
from transformers import pipeline
import mysql.connector
import chromadb

In [2]:
%pip install --upgrade --quiet  langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_google_genai import GoogleGenerativeAI

In [4]:
from getpass import getpass

api_key = getpass()

 ········


In [5]:
llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=api_key)

In [6]:
#Testing LLM
response = llm.invoke("Write me a poem on anime")
print(response)

**Anime**

Anime, oh anime,
You bring me such joy.
Your stories are so engaging,
I can't help but watch all day.

I love the characters,
So unique and relatable.
I love the worlds they live in,
So full of wonder and adventure.

Anime has taught me so much,
About life, friendship, and love.
It has helped me to grow as a person,
And to become a better version of myself.

Thank you, anime,
For everything.
You are my one true love,
And I will never forget you.

I will continue to watch your shows,
And read your manga,
For as long as I live.

Anime, you are my everything.


In [7]:
question = """3 rows from discounts table:
id	shirt_id	pct_discount
1	1	          10.00
2	2	          15.00
3	3	          20.00
What is the discount on the discount_id=3?
"""
response =llm.invoke(question)
print(response)

20.00


In [9]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [10]:
db_user = "root"
db_password = "root"
db_host = "localhost"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [11]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
ans1 = db_chain.invoke("How many t-shirts do we have left for nike in extra small size and white color?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(45,)]
Answer:45
> Finished chain.


In [12]:
print(ans1)

{'query': 'How many t-shirts do we have left for nike in extra small size and white color?', 'result': '45'}


In [13]:
# ans1 is correct answer

In [14]:
ans2 = db_chain.invoke("How much is the price of the inventory for all small size t-shirts?")
print(ans2)



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('432'),)]
Answer:432
> Finished chain.
{'query': 'How much is the price of the inventory for all small size t-shirts?', 'result': '432'}


In [15]:
# ans2 is the wrong answer as we require to do SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' since inventory total item price = stock_quantity*price

In [16]:
ans2 = db_chain.invoke("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('23115'),)]
Answer:23115
> Finished chain.


In [17]:
# This is the correct answer

In [18]:
ans3 = db_chain.invoke("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


SELECT SUM(price * (1 - pct_discount))
FROM t_shirts AS t1
INNER JOIN discounts AS t2 ON t1.t_shirt_id = t2.t_shirt_id
WHERE t1.brand = 'Levi'
AND t1.stock_quantity > 0
AND CURDATE() BETWEEN t2.start_date AND t2.end_date

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 't2.start_date' in 'where clause'")
[SQL: SELECT SUM(price * (1 - pct_discount))
FROM t_shirts AS t1
INNER JOIN discounts AS t2 ON t1.t_shirt_id = t2.t_shirt_id
WHERE t1.brand = 'Levi'
AND t1.stock_quantity > 0
AND CURDATE() BETWEEN t2.start_date AND t2.end_date]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [19]:
#Error due to unfamiliarity of database 

In [20]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

ans3 = db_chain.invoke(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('26272.200000'),)]
Answer:

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


26272.2
> Finished chain.


In [22]:
#Correct Answer for THE SQL Query

In [23]:
ans4 = db_chain.invoke("How many black color Levi's t shirts we have available whose size is bigger than small?")



> Entering new SQLDatabaseChain chain...
How many black color Levi's t shirts we have available whose size is bigger than small?
SQLQuery:SELECT count(*) FROM t_shirts WHERE brand = 'Levi' AND color = 'Black' AND size > 'S'
SQLResult: [(2,)]
Answer:2
> Finished chain.


In [25]:
#Wrong answer as it did not get the quantity and only the number of records

In [26]:
ans5 = db_chain.invoke("How much stock of black color Levi's t shirts we have available whose size is bigger than small?")



> Entering new SQLDatabaseChain chain...
How much stock of black color Levi's t shirts we have available whose size is bigger than small?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE color='Black' AND brand='Levi' AND size > 'S'
SQLResult: [(68,), (55,)]
Answer:68
> Finished chain.


In [27]:
#Wrong answer as the query is slightly wrong and also no sum operation is performed

In [28]:
ans5 = db_chain.invoke("""SELECT SUM(stock_quantity) FROM t_shirts where color="Black" and brand = "Levi" and size IN ("M","L","XL");""")



> Entering new SQLDatabaseChain chain...
SELECT SUM(stock_quantity) FROM t_shirts where color="Black" and brand = "Levi" and size IN ("M","L","XL");
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts where color="Black" and brand = "Levi" and size IN ("M","L","XL");
SQLResult: [(Decimal('143'),)]
Answer:143
> Finished chain.


In [29]:
#This is the correct answer using SQL query

In [34]:
ans6 = db_chain("How many white color Levi's shirt I have?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's shirt I have?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(81,), (36,), (25,), (63,)]
Answer:81
> Finished chain.


In [35]:
ans6 = db_chain.invoke("SELECT SUM(stock_quantity) FROM t_shirts WHERE color = 'White' AND brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(stock_quantity) FROM t_shirts WHERE color = 'White' AND brand = 'Levi'
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts WHERE color = 'White' AND brand = 'Levi'
SQLResult: [(Decimal('205'),)]
Answer:

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


205
> Finished chain.


*FEW SHOT LEARNING*

In [36]:
few_shots = [
    {
        'Question': "How many t-shirts do we have left for Nike in XS size and white color?",
        'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
        'SQLResult': "Result of the SQL query",
        'Answer': ans1['result']
    },
    {
        'Question': "How much is the total price of the inventory for all S-size t-shirts?",
        'SQLQuery': "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
        'SQLResult': "Result of the SQL query",
        'Answer': ans2['result']
    },
    {
        'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?",
        'SQLQuery': """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue 
                       FROM (SELECT sum(price*stock_quantity) as total_amount, t_shirt_id 
                             FROM t_shirts 
                             WHERE brand = 'Levi'
                             GROUP BY t_shirt_id) a 
                       LEFT JOIN discounts ON a.t_shirt_id = discounts.t_shirt_id""",
        'SQLResult': "Result of the SQL query",
        'Answer': ans3['result']
    },
    {
        'Question': "How many black color Levi's t-shirts do we have available whose size is bigger than small?",
        'SQLQuery': "SELECT SUM(stock_quantity) FROM t_shirts WHERE color='Black' AND brand = 'Levi' AND size IN ('M','L','XL')",
        'SQLResult': "Result of the SQL query",
        'Answer': ans5['result']
    },
    {
        'Question': "How much stock of black color Levi's t-shirts do we have available whose size is bigger than small?",
        'SQLQuery': "SELECT SUM(stock_quantity) FROM t_shirts WHERE color='Black' AND brand = 'Levi' AND size IN ('M','L','XL')",
        'SQLResult': "Result of the SQL query",
        'Answer': ans5['result']
    },
    {
        'Question': "How many white color Levi's shirt I have?",
        'SQLQuery': "SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
        'SQLResult': "Result of the SQL query",
        'Answer': ans6['result']
    }
    
]


In [39]:
#!pip install -U langchain-huggingface

In [40]:

from langchain.prompts import SemanticSimilarityExampleSelector
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

to_vectorize = [" ".join(example.values()) for example in few_shots]

In [41]:
to_vectorize

["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 45",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query 23115",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue \n                       FROM (SELECT sum(price*stock_quantity) as total_amount, t_shirt_id \n                             FROM t_shirts \n                             WHERE brand = 'Levi'\n                             GROUP BY t_shirt_id) a \n                       LEFT JOIN discounts ON a.t_shirt_id = discounts.t_shirt_id Result of the SQL query 26272.2",
 "How many black color Levi's t-shirts do we 

In [42]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [44]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "How many black color NIKE shirts do we have that are bigger than Large?"})

[{'Answer': '143',
  'Question': "How many black color Levi's t-shirts do we have available whose size is bigger than small?",
  'SQLQuery': "SELECT SUM(stock_quantity) FROM t_shirts WHERE color='Black' AND brand = 'Levi' AND size IN ('M','L','XL')",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '45',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'}]

In [81]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today". Also When ASKING for stock of multiple objects use the SUM(). The size list is given as [XS,S,M,L,XL] in increasing order of size.
So when asked to compare sizes instead of using operators like >,<,=, use the IN() statement instead.
Also if you are not sure about the answer, just don't return the wrong answer and write that you do not have much information about it. Make sure that you are using joins properly when asked such queries which require information from 2 or more tables.

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [82]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [83]:
#This prompt suffix is fine and workable

In [84]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [85]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"],
)

In [86]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [87]:
better_ans3 = new_chain.invoke("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)
SQLQuery:SELECT sum(price*stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('26347'),)]
Answer:

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


26347
> Finished chain.


In [88]:
better_ans2 = new_chain.invoke("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT sum(price*stock_quantity) FROM t_shirts WHERE size='S'
SQLResult: [(Decimal('23115'),)]
Answer:23115
> Finished chain.


In [89]:
better_ans4and5 = new_chain.invoke("How many black color Levi's t shirts we have available whose size is bigger than small?")



> Entering new SQLDatabaseChain chain...
How many black color Levi's t shirts we have available whose size is bigger than small?
SQLQuery:

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


SELECT SUM(stock_quantity) FROM t_shirts WHERE color='Black' AND brand = 'Levi' AND size IN ('M','L','XL')
SQLResult: [(Decimal('143'),)]
Answer:143
> Finished chain.


In [90]:
#Lets ask another similar question to this 

In [91]:
better_ans_alter = new_chain.invoke("How many white color Nike t-shirts we have available whose size is bigger than medium?")



> Entering new SQLDatabaseChain chain...
How many white color Nike t-shirts we have available whose size is bigger than medium?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size > 'M'
SQLResult: [(Decimal('45'),)]
Answer:45
> Finished chain.


In [92]:
#The answer is correct and we can see that our llm works good on our SQL database.

In [93]:
#Lets ask a SQL query with Join from our t_shirt table to discount table

In [95]:
final_ans = new_chain.invoke("Show me the first 5 records those t_shirts which have a discount associated with them along with the percentage discount applied on their price.")



> Entering new SQLDatabaseChain chain...
Show me the first 5 records those t_shirts which have a discount associated with them along with the percentage discount applied on their price.
SQLQuery:SELECT t_shirts.t_shirt_id, t_shirts.brand, t_shirts.color, t_shirts.size, t_shirts.price, discounts.pct_discount 
FROM discounts 
INNER JOIN t_shirts ON discounts.t_shirt_id = t_shirts.t_shirt_id 
ORDER BY t_shirts.t_shirt_id ASC LIMIT 5
SQLResult: [(1, 'Levi', 'Black', 'XS', 11, Decimal('10.00')), (2, 'Van Huesen', 'White', 'S', 28, Decimal('15.00')), (3, 'Nike', 'White', 'M', 22, Decimal('20.00')), (4, 'Nike', 'Black', 'XL', 28, Decimal('5.00')), (5, 'Nike', 'Blue', 'L', 22, Decimal('25.00'))]
Answer:[(1, 'Levi', 'Black', 'XS', 11, Decimal('10.00')), (2, 'Van Huesen', 'White', 'S', 28, Decimal('15.00')), (3, 'Nike', 'White', 'M', 22, Decimal('20.00')), (4, 'Nike', 'Black', 'XL', 28, Decimal('5.00')), (5, 'Nike', 'Blue', 'L', 22, Decimal('25.00'))]
> Finished chain.


In [96]:
#Another correct query answer, our llm has become really good with performing DML queries with our database.